In [1]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/shopping_trends_updated']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
exec(os.environ['IREWR_IMPORTS'])

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# FIRST-AUTHOR: remove path printing
# import os
# for dirname, _, filenames in os.walk('./input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


## Loading the data

In [3]:
df = pd.read_csv("./input/shopping_trends_updated.scaled.csv")
df.head()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/26 22:18:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/26 22:18:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/26 22:18:10 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/01/26 22:18:10 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/01/26 22:18:10 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/01/26 22:18:10 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause addition

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


## Shape of the dataset (Total number of rows and columns in the dataset)

In [4]:
df.shape

(8190, 18)

## Columns present in the dataset

In [5]:
df.columns

Index(['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category',
       'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season',
       'Review Rating', 'Subscription Status', 'Shipping Type',
       'Discount Applied', 'Promo Code Used', 'Previous Purchases',
       'Payment Method', 'Frequency of Purchases'],
      dtype='object')

## Looking at the brief information about the dataset

In [6]:
df.info()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 8190 entries, 0 to 8189
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             8190 non-null   int32  
 1   Age                     8190 non-null   int32  
 2   Gender                  8190 non-null   object 
 3   Item Purchased          8190 non-null   object 
 4   Category                8190 non-null   object 
 5   Purchase Amount (USD)   8190 non-null   int32  
 6   Location                8190 non-null   object 
 7   Size                    8190 non-null   object 
 8   Color                   8190 non-null   object 
 9   Season                  8190 non-null   object 
 10  Review Rating           8190 non-null   float64
 11  Subscription Status     8190 non-null   object 
 12  Shipping Type           8190 non-null   object 
 13  Discount Applied        8190 non-null   object 
 14  Promo Code Used         8190 non-null

## Checking for NULL values

In [7]:
evaluate_lazy(df.isnull().sum())

Customer ID               0
Age                       0
Gender                    0
Item Purchased            0
Category                  0
Purchase Amount (USD)     0
Location                  0
Size                      0
Color                     0
Season                    0
Review Rating             0
Subscription Status       0
Shipping Type             0
Discount Applied          0
Promo Code Used           0
Previous Purchases        0
Payment Method            0
Frequency of Purchases    0
dtype: int64

## Checking for duplicate value/rows

In [8]:
evaluate_lazy(df.duplicated().sum())

4290

## There are no NULL values and neither are any Duplicated rows.
### Moving on with the EDA

In [9]:
evaluate_lazy(df.describe())

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
25/01/26 22:18:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,8190.000000,8190.000000,8190.000000,8190.000000,8190.000000
mean,1866.928571,44.093162,59.742369,3.750427,25.404396
std,1160.859406,15.214580,23.685798,0.715657,14.452598
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,829.000000,31.000000,39.000000,3.100000,13.000000
50%,1852.000000,44.000000,60.000000,3.700000,25.000000
75%,2876.000000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


## Insights :
* The minimum age of customer in the given dataset is **18**, while the maximum age is **70** and the average age of the customers is **44**.
* The average Purchase Amount (USD) is **59.7**
* The average Review Rating is **3.7** 

In [10]:
# FIRST-AUTHOR: remove plotting
# import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px

# %matplotlib inline

## Distribution of age groups of the customers.

In [11]:
# FIRST-AUTHOR: remove plotting
# fig = px.histogram(df, x="Age", nbins= 20, marginal="rug",text_auto = True)
# fig.show()

## Gender Distribution of Customers

In [12]:
gender_counts = df['Gender'].value_counts()
# FIRST-AUTHOR: remove plotting
# fig = px.pie(data_frame=df, names=gender_counts.index, values=gender_counts.values)
# fig.update_traces(textposition='inside', textinfo='percent+label')
# fig.show()
_ = gender_counts.index
_ = gender_counts.values

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


## What is the Purchase Distribution as per Category?


In [13]:
category_data = df['Category'].value_counts()
# FIRST-AUTHOR: remove plotting
# fig = px.pie(data_frame=df, names=category_data.index, values=category_data.values)
# fig.update_traces(textposition='inside', textinfo='percent+label')
# fig.show()
_ = category_data.index
_ = category_data.values


/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [14]:
df.columns

Index(['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category',
       'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season',
       'Review Rating', 'Subscription Status', 'Shipping Type',
       'Discount Applied', 'Promo Code Used', 'Previous Purchases',
       'Payment Method', 'Frequency of Purchases'],
      dtype='object')

## Wich is the Highest and the Lowest selling Item ?

In [15]:
sold_items = df['Item Purchased'].value_counts().sort_values(ascending = False)

# FIRST-AUTHOR: remove plotting
# fig = px.bar(sold_items, x = sold_items.index, y = sold_items.values, 
#              title="Highest selling Items", labels={"y":"Quantity"})
# fig.show()
_ = sold_items.index
_ = sold_items.values


/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


## Items Purchased by respective Genders

In [16]:
var = df.groupby(['Item Purchased', 'Gender']).size().reset_index(name='Customer Count')
sorted_var = var.sort_values(by='Item Purchased', ascending=False)

# FIRST-AUTHOR: remove plotting
# fig = px.bar(sorted_var, 
#              x='Item Purchased', 
#              y='Customer Count', 
#              color='Gender',
#              title='Most Purchased Items by Gender',
#              labels={'Customer Count': 'Count of Customers'})

# fig.update_traces(hovertemplate='Item=%{x}<br>Count of Customers=%{y}', 
#                   texttemplate='%{y}', 
#                   textposition='inside')

# fig.update_layout(yaxis_title='Count of Customers')

# fig.show()


## Which category generates the maximum sales?

In [17]:
var = df.groupby('Category')['Purchase Amount (USD)'].sum().reset_index().sort_values(by='Purchase Amount (USD)',ascending=False)

# FIRST-AUTHOR: remove plotting
# fig = px.bar(var, x='Category', y = 'Purchase Amount (USD)')

# fig.show()

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/groupby.py:893: FutureWarning: Default value of `numeric_only` will be changed to `False` instead of `True` in 4.0.0.
  warnings.warn(


## Which TOP 10 locations generates the Maximum sales?

In [18]:
var = df.groupby('Location')['Purchase Amount (USD)'].sum().reset_index().sort_values(by='Purchase Amount (USD)',ascending=False).head(10)

# FIRST-AUTHOR: remove plotting
# fig = px.bar(var, x='Location', y = 'Purchase Amount (USD)')

# fig.show()

## Which Size was mostly preffered by the customers?

In [19]:
size_count = df['Size'].value_counts()

# FIRST-AUTHOR: remove plotting
# fig = px.bar(size_count, x=size_count.index, y=size_count.values, labels = {'y':'Number of Occurances'})
# fig.show()
_ = size_count.index
_ = size_count.values

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


## What TOP 10 colors were preffered by the customers?

In [20]:
color_count = df['Color'].value_counts().head(10)

# FIRST-AUTHOR: remove plotting
# fig = px.bar(color_count, x = color_count.index, y = color_count.values, labels = {'y':'Number of Occurances'})
# fig.show()
_ = color_count.index
_ = color_count.values

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [21]:
df.columns

Index(['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category',
       'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season',
       'Review Rating', 'Subscription Status', 'Shipping Type',
       'Discount Applied', 'Promo Code Used', 'Previous Purchases',
       'Payment Method', 'Frequency of Purchases'],
      dtype='object')

## Number of customers distribution over the different seasons

In [22]:

season_data = df['Season'].value_counts()
# FIRST-AUTHOR: remove plotting
# fig = px.pie(data_frame=df, names=season_data.index, values=season_data.values)
# fig.update_traces(textposition='inside', textinfo='percent+label')
# fig.show()
_ = season_data.index
_ = season_data.values


/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


## How many customers had the subscription?

In [23]:
subs_count = df['Subscription Status'].value_counts()

# FIRST-AUTHOR: remove plotting
# fig = px.bar(subs_count, x = subs_count.index, y = subs_count.values, labels = {'y':'Number of Customers'})
# fig.show()
_ = subs_count.index
_ = subs_count.values

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [24]:
shipping_count = df['Shipping Type'].value_counts()

# FIRST-AUTHOR: remove plotting
# fig = px.bar(shipping_count, x = shipping_count.index, y = shipping_count.values, labels = {'y':'Shipped packages'})
# fig.show()
_ = shipping_count.index
_ = shipping_count.values

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [25]:
disc_count = df['Discount Applied'].value_counts()

# FIRST-AUTHOR: remove plotting
# fig = px.bar(disc_count, x = disc_count.index, y = disc_count.values, labels = {'y':'Number of customers'})
# fig.show()
_ = disc_count.index
_ = disc_count.values

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [26]:
df.columns

Index(['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category',
       'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season',
       'Review Rating', 'Subscription Status', 'Shipping Type',
       'Discount Applied', 'Promo Code Used', 'Previous Purchases',
       'Payment Method', 'Frequency of Purchases'],
      dtype='object')

In [27]:
evaluate_lazy(df['Promo Code Used'].value_counts())

/home/user/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(


No     4446
Yes    3744
Name: Promo Code Used, dtype: int64